Isthmus / Mitchellpkt

March 2020

## Import libraries

In [1]:
import numpy as np
import isthmuslib as isli
import matplotlib as pyplot 
import random as random

## Parameters

In [2]:
fractional_difficulty = 0.999

## Mimic a PoW iteration

In [3]:
def hash_attempt_boolean(fractional_difficulty):
    random_num = random.random()
    if random_num >= fractional_difficulty:
        return True
    else:
        return False

## Define worker

In [34]:
def get_initial_nonce(search_strategy, nonce_range):
    # Pick initial nonce
    if search_strategy == 'increment_from_0':
        return 0
    elif search_strategy == 'increment_from_halfway':
        return int(np.floor(nonce_range / 2))
    elif search_strategy == 'decrement_from_top':
        return int(nonce_range)
    elif search_strategy == 'uniform':
        return int(np.floor(random.random()*nonce_range))


class Worker:

    def __init__(self, hashes_per_second=100, search_strategy = 'increment_from_0', nonce_range = 2**32, workerID = None):
        self.hashes_per_second = hashes_per_second
        self.found_block_yet = False
        self.search_strategy = search_strategy
        self.nonce_range = nonce_range
        self.nonce = get_initial_nonce(search_strategy, nonce_range)
        if workerID is None:
            self.workerID = np.floor(random.random()*1000000)
        else:
            self.workerID = workerID

    def reset(self):
        self.found_block_yet = False
        self.nonce = get_initial_nonce(self.search_strategy, self.nonce_range)

        # Pick reset nonce
        if self.search_strategy == 'increment_from_0':
            self.nonce = 0
        elif self.search_strategy == 'increment_from_halfway':
            self.nonce = np.floor(self.nonce_range / 2)
        elif self.search_strategy == 'decrement_from_top':
            self.nonce = np.floor(self.nonce_range)
        elif self.search_strategy == 'uniform':
            self.nonce = int(np.floor(random.random()*self.nonce_range))

    def single_hash(self, fractional_difficulty, verbose = False):
        
        # Make an attempt
        attempt_result = hash_attempt_boolean(fractional_difficulty)
        if attempt_result:
            self.found_block_yet = True
            
        # Update the nonce
        if (self.search_strategy == 'increment_from_0') or (self.search_strategy == 'increment_from_halfway'):
            self.nonce += 1
        elif self.search_strategy == 'decrement_from_top':
            self.nonce -= 1
        elif self.search_strategy == 'uniform':
            self.nonce = int(np.floor(random.random()*self.nonce_range))
        else:
            return NameError('Search strategy not recognized')
            
        return attempt_result
    
    def hash_n_times(self, number_of_hashes, fractional_difficulty, verbose = False):
        iteration = 0
        while (self.found_block_yet == False)& (iteration <= number_of_hashes):
            self.single_hash(fractional_difficulty)
            if verbose:
                print('--------------')
                print('Nonce: ' + str (self.nonce))
                print('Step iteration: ' + str(iteration) + ' of ' + str(number_of_hashes))
                print('Successful? ' + str(self.found_block_yet))
            iteration += 1
        return self.found_block_yet
            
    def hash_n_seconds(self, number_of_seconds, fractional_difficulty, verbose = False):
        number_of_hashes = number_of_seconds * self.hashes_per_second
        self.hash_n_times(number_of_hashes, fractional_difficulty, verbose = verbose)
        return self.found_block_yet

    def print(self):
        print('---------------')
        print('Worker hashrate: ' + str(self.hashes_per_second) + ' H/s')
        print('Search strategy: '+ self.search_strategy)
        print('Nonce range: ' + str(self.nonce_range))
        print('Current nonce: ' + str(self.nonce))
        print('Found block yet: ' + str(self.found_block_yet))

In [44]:
class Network:
    def __init__(self, height=0, fractional_difficulty=0.999):
        self.height = height
        self.fractional_difficulty = fractional_difficulty
        self.block_height_list = list()
        self.block_nonce_list = list()
        self.worker_objects = list()

    def initialize_n_workers(self, number_of_workers=100, worker_hashes_per_second=10, search_strategy = 'increment_from_0', nonce_range = 2**32):
        for n in range(number_of_workers):
            self.worker_objects.append(Worker(hashes_per_second=worker_hashes_per_second, search_strategy = search_strategy, nonce_range = nonce_range))

    def mine_1_block(self, timestep_seconds = 1, verbose = False):
        found_block = False
        nth_step = 0
        while not found_block:
            if verbose:
                print('$$$$$$$$$$ nth_step #: ' + str(nth_step))
            for count, worker_object in enumerate(self.worker_objects):
                if nth_step == 0:
                    worker_object.reset()
                result = worker_object.hash_n_seconds(timestep_seconds, self.fractional_difficulty, verbose = verbose)
                # Do I have to put the object back??
                self.worker_objects[count] = worker_object
                if verbose:
                    print('**********************')
                    print('Count: ' + str(count) + ' // workerID: ' + str(worker_object.workerID))
                    print("Result: " + str(result))
                if result:
                    found_block = True
                    self.block_height_list.append(self.height)
                    self.block_nonce_list.append(worker_object.nonce)
                    self.height += 1
                    break
                else:
                    nth_step += 1

    def print(self, level=1):
        print("--------")
        print("Height: " + str(self.height))
        print("Fractional difficulty: "  + str(self.fractional_difficulty))
        print("Number of blocks: " + str(len(self.block_height_list)))
        print("Number of workers: " + str(len(self.worker_objects)))
        if level > 1:
            for h in range(len(self.block_height_list)):
                print('Height: ' + str(self.block_height_list[h]) + ' | Nonce: ' +  str(self.block_nonce_list[h]))

In [45]:
N = Network(fractional_difficulty=0.99)
N.print()

--------
Height: 0
Fractional difficulty: 0.99
Number of blocks: 0
Number of workers: 0


In [46]:
N.initialize_n_workers(number_of_workers=2,worker_hashes_per_second=5)
N.print()

--------
Height: 0
Fractional difficulty: 0.99
Number of blocks: 0
Number of workers: 2


In [51]:
N.mine_1_block(verbose=True)
N.print(2)

$$$$$$$$$$ nth_step #: 0
--------------
Nonce: 1
Step iteration: 0 of 5
Successful? False
--------------
Nonce: 2
Step iteration: 1 of 5
Successful? False
--------------
Nonce: 3
Step iteration: 2 of 5
Successful? False
--------------
Nonce: 4
Step iteration: 3 of 5
Successful? False
--------------
Nonce: 5
Step iteration: 4 of 5
Successful? False
--------------
Nonce: 6
Step iteration: 5 of 5
Successful? False
**********************
Count: 0 // workerID: 33215.0
Result: False
--------------
Nonce: 7
Step iteration: 0 of 5
Successful? False
--------------
Nonce: 8
Step iteration: 1 of 5
Successful? False
--------------
Nonce: 9
Step iteration: 2 of 5
Successful? False
--------------
Nonce: 10
Step iteration: 3 of 5
Successful? False
--------------
Nonce: 11
Step iteration: 4 of 5
Successful? False
--------------
Nonce: 12
Step iteration: 5 of 5
Successful? False
**********************
Count: 1 // workerID: 445722.0
Result: False
$$$$$$$$$$ nth_step #: 2
--------------
Nonce: 7
Step ite

--------
Height: 3
Fractional difficulty: 0.99
Number of blocks: 3
Number of workers: 2
Height: 0 | Nonce: 12
Height: 1 | Nonce: 2
Height: 2 | Nonce: 9
